In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from xgboost.sklearn import XGBClassifier

Classificando digitos escritos a mão utilizando diversos modelos populares (DecisionTree, RandomForest, XGBoost, SVM) e ao final utilizando dois modelos de rede neural desenvolvido em Pytorch.

# Obtendo os dados

In [2]:
data = fetch_openml("mnist_784", version=1)
print(data)

{'data':        pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
69995       0       0       0       0       0       0       0       0       0   
69996       0       0       0       0       0       0       0       0       0   
69997       0       0       0       0       0       0       0       0       0   
69998       0       0       0       0       0       0       0       0       0   
69999       0       0       0       0       0       0       0       0       0   

       pixel10  ..

In [3]:
x, y = data["data"], data["target"]
print(x.shape)

(70000, 784)


## Dividindo os dados para os testes

In [4]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, 
                                                test_size=0.2,
                                                random_state=42)

# Treinando os modelos

## Treinando Modelos Prontos

In [5]:
models = []
accuracies = []

In [6]:
model = SGDClassifier()
model.fit(xtrain, ytrain)
predictions = model.predict(xtest)
accuracy = accuracy_score(ytest, predictions)
models.append(model)
accuracies.append(accuracy)
print(f"Accuracy of the SGDClassifier classifier is: {accuracy}")

Accuracy of the SGDClassifier classifier is: 0.8877857142857143


In [7]:
model = RandomForestClassifier()
model.fit(xtrain, ytrain)
predictions = model.predict(xtest)
accuracy = accuracy_score(ytest, predictions)
models.append(model)
accuracies.append(accuracy)
print(f"Accuracy of the RandomForestClassifier classifier is: {accuracy}")
# Calculating the accuracy of classifier


Accuracy of the RandomForestClassifier classifier is: 0.9682857142857143


In [8]:
model = DecisionTreeClassifier()
model.fit(xtrain, ytrain)
predictions = model.predict(xtest)
accuracy = accuracy_score(ytest, predictions)
models.append(model)
accuracies.append(accuracy)
print(f"Accuracy of the DecisionTreeClassifier classifier is: {accuracy}")

Accuracy of the DecisionTreeClassifier classifier is: 0.8707142857142857


In [9]:
model = SVC()
model.fit(xtrain, ytrain)
predictions = model.predict(xtest)
accuracy = accuracy_score(ytest, predictions)
models.append(model)
accuracies.append(accuracy)
print(f"Accuracy of the SVC classifier is: {accuracy}")

Accuracy of the SVC classifier is: 0.9764285714285714


In [10]:
model = KNeighborsClassifier()
model.fit(xtrain.values, ytrain)
predictions = model.predict(xtest.values)
accuracy = accuracy_score(ytest, predictions)
models.append(model)
accuracies.append(accuracy)
print(f"Accuracy of the KNeighborsClassifier classifier is: {accuracy}")

Accuracy of the KNeighborsClassifier classifier is: 0.9700714285714286


In [11]:
model = XGBClassifier()
ytrain = ytrain.astype('int64')
model.fit(xtrain, ytrain)
predictions = model.predict(xtest)
ytrain = ytrain.astype('str')
predictions = predictions.astype('str')
accuracy = accuracy_score(ytest, predictions)
models.append(model)
accuracies.append(accuracy)
print(f"Accuracy of the XGBClassifier classifier is: {accuracy}")

Accuracy of the XGBClassifier classifier is: 0.9780714285714286


In [12]:
best_model = models[accuracies.index(max(accuracies))]
best_model.__class__

xgboost.sklearn.XGBClassifier

## Redes Modeladas

Serão contruidas duas redes: 

    Uma Simples, com apenas o layer de input, um hidden layer e o output player, com funções de ativação sigmoidal. 
    Outra rede mais complexa, com 2 layers convolucionais, 1 hidden layer e o output layer, com função de ativação RELU e a utilização de dropout. 

Os dados de treinamento serão adicionados para visualização utilizando o tensorboard e os modelos serão salvos para posterior utilização.

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import math

### Rede Simples

In [6]:
class SimpleNet (nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNet,self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=-1)
        # return x

In [7]:
#Cria o modelo, otimizador e a função perda
input_size = xtrain.shape[1]
output_size = ytrain.unique().shape[0]

In [8]:
xtrain_tensor = torch.from_numpy(xtrain.values).to(torch.float32)
xtest_tensor = torch.from_numpy(xtest.values).to(torch.float32)
ytrain_tensor = torch.from_numpy(np.array(np.array(ytrain.values.astype(int)))).type(torch.LongTensor)
ytest_tensor = torch.from_numpy(np.array(np.array(ytest.values.astype(int)))).type(torch.LongTensor)

In [9]:
def train(model, xtrain_data, ytrain_data, mini_batch_size, loss_fn, optimizer, epoch, tb_writer):
    model.train(True)

    running_loss = 0.
    last_loss = 0.
    correct = 0
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch

    batch_ids = math.ceil(len(xtrain_data)/mini_batch_size)

    # index and do some intra-epoch reporting
    for i in range(0, batch_ids):
        # get the input and labels for this particular batch
        inputs = xtrain_data[i*mini_batch_size:(i+1)*mini_batch_size-1]
        labels = ytrain_data[i*mini_batch_size:(i+1)*mini_batch_size-1]

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()
        # Gather data and report
        pred = outputs.argmax(dim=-1, keepdim=True)
        correct += pred.eq(labels.view_as(pred)).sum().item()
        running_loss += loss.item()
        if (i) % 1000 == 0 and i != 0:
            last_loss = running_loss / (1000*mini_batch_size) # loss per batch
            print(f'Train Epoch: {epoch} [{i*mini_batch_size}/{len(xtrain_data)} ({100*i*mini_batch_size/len(xtrain_data):.0f}%)]\tLoss: {last_loss:.6f}')
            tb_x = (epoch-1) * len(xtrain_data) + i*mini_batch_size + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    correct = 0
    outputs = model(xtrain_data)
    pred = outputs.argmax(dim=-1, keepdim=True)
    correct += pred.eq(ytrain_data.view_as(pred)).sum().item()
    
    accuracy = correct/len(ytrain_data)
    tb_writer.add_scalar('Accuracy x epoch', accuracy, epoch)
    return last_loss, accuracy

def test(model, xtest_data, ytest_data, loss_fn):
    model.eval()
    running_vloss = 0.0
    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        correct = 0
        voutputs = model(xtest_data)
        vpred = voutputs.argmax(dim=-1, keepdim=True)
        correct += vpred.eq(ytest_data.view_as(vpred)).sum().item()
        vloss = loss_fn(voutputs, ytest_data)
        running_vloss += vloss

    avg_vloss = running_vloss / len(xtest_data)
    accuracy = correct/len(ytest_data)

    return avg_vloss, accuracy
   

In [15]:
model = SimpleNet(input_size, 128, 10)
optimizer = optim.Adam(model.parameters())
loss_fn = nn.NLLLoss(reduction='sum')

In [16]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter(f'runs/mnist_trainer_{model.__class__.__name__}_{timestamp}')

EPOCHS = 50

best_vloss = 1_000_000.

for epoch in range(1, EPOCHS+1):

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss, accuracy = train(model, xtrain_tensor, ytrain_tensor, 5, loss_fn, optimizer, epoch, writer)
    
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()
    avg_vloss, vaccuraccy = test(model, xtest_tensor, ytest_tensor, loss_fn)
    
    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch)
    writer.add_scalars('Training vs. Validation Accuracy',
                    { 'Training' : accuracy, 'Validation' : vaccuraccy },
                    epoch)
    writer.flush()

    print(f'EPOCH {epoch}: Training Accuracy: {accuracy*100:.0f}  Validation Accuracy: {vaccuraccy*100:.0f}')
    # Track best performance, and save the model's state
    
    model_path = f'./models/{model.__class__.__name__}_{timestamp}_{epoch}'
    torch.save(model.state_dict(), model_path)

    

Train Epoch: 1 [5000/56000 (9%)]	Loss: 1.102252
Train Epoch: 1 [10000/56000 (18%)]	Loss: 0.638253
Train Epoch: 1 [15000/56000 (27%)]	Loss: 0.522308
Train Epoch: 1 [20000/56000 (36%)]	Loss: 0.481568
Train Epoch: 1 [25000/56000 (45%)]	Loss: 0.464815
Train Epoch: 1 [30000/56000 (54%)]	Loss: 0.412057
Train Epoch: 1 [35000/56000 (62%)]	Loss: 0.395170
Train Epoch: 1 [40000/56000 (71%)]	Loss: 0.404641
Train Epoch: 1 [45000/56000 (80%)]	Loss: 0.391435
Train Epoch: 1 [50000/56000 (89%)]	Loss: 0.392673
Train Epoch: 1 [55000/56000 (98%)]	Loss: 0.367541
EPOCH 1: Training Accuracy: 85  Validation Accuracy: 85
Train Epoch: 2 [5000/56000 (9%)]	Loss: 0.358158
Train Epoch: 2 [10000/56000 (18%)]	Loss: 0.335753
Train Epoch: 2 [15000/56000 (27%)]	Loss: 0.334808
Train Epoch: 2 [20000/56000 (36%)]	Loss: 0.320901
Train Epoch: 2 [25000/56000 (45%)]	Loss: 0.337430
Train Epoch: 2 [30000/56000 (54%)]	Loss: 0.293671
Train Epoch: 2 [35000/56000 (62%)]	Loss: 0.294536
Train Epoch: 2 [40000/56000 (71%)]	Loss: 0.30772

### Rede Complexa

In [17]:
class ComplexNet(nn.Module):
    def __init__(self):
        super(ComplexNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [18]:
xtrain_tensor = xtrain_tensor.reshape(-1, 1, 28, 28)
xtest_tensor = xtest_tensor.reshape(-1, 1, 28, 28)


In [19]:
model = ComplexNet()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.NLLLoss(reduction='sum')

In [20]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter(f'runs/mnist_trainer_{model.__class__.__name__}_{timestamp}')

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(1, EPOCHS+1):

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss, accuracy = train(model, xtrain_tensor, ytrain_tensor, 5, loss_fn, optimizer, epoch, writer)
    
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()
    avg_vloss, vaccuraccy = test(model, xtest_tensor, ytest_tensor, loss_fn)
    
    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch)
    writer.add_scalars('Training vs. Validation Accuracy',
                    { 'Training' : accuracy, 'Validation' : vaccuraccy },
                    epoch)
    writer.flush()

    print(f'EPOCH {epoch}: Training Accuracy: {accuracy*100:.0f}  Validation Accuracy: {vaccuraccy*100:.0f}')
    # Track best performance, and save the model's state
    
    model_path = f'./models/{model.__class__.__name__}_{timestamp}_{epoch}'
    torch.save(model.state_dict(), model_path)

Train Epoch: 1 [5000/56000 (9%)]	Loss: 0.820871
Train Epoch: 1 [10000/56000 (18%)]	Loss: 0.355651
Train Epoch: 1 [15000/56000 (27%)]	Loss: 0.291579
Train Epoch: 1 [20000/56000 (36%)]	Loss: 0.271906
Train Epoch: 1 [25000/56000 (45%)]	Loss: 0.238396
Train Epoch: 1 [30000/56000 (54%)]	Loss: 0.216273
Train Epoch: 1 [35000/56000 (62%)]	Loss: 0.216513
Train Epoch: 1 [40000/56000 (71%)]	Loss: 0.215480
Train Epoch: 1 [45000/56000 (80%)]	Loss: 0.190822
Train Epoch: 1 [50000/56000 (89%)]	Loss: 0.218021
Train Epoch: 1 [55000/56000 (98%)]	Loss: 0.194192
EPOCH 1: Training Accuracy: 93  Validation Accuracy: 97
Train Epoch: 2 [5000/56000 (9%)]	Loss: 0.190083
Train Epoch: 2 [10000/56000 (18%)]	Loss: 0.190492
Train Epoch: 2 [15000/56000 (27%)]	Loss: 0.171495
Train Epoch: 2 [20000/56000 (36%)]	Loss: 0.197563
Train Epoch: 2 [25000/56000 (45%)]	Loss: 0.166390
Train Epoch: 2 [30000/56000 (54%)]	Loss: 0.160300
Train Epoch: 2 [35000/56000 (62%)]	Loss: 0.177236
Train Epoch: 2 [40000/56000 (71%)]	Loss: 0.15759